# Comparing OLS and Double Selection
This material is originally developed for ACE 592 Big Data - Empirical Economics written on 7/21/2017 by Alex Stevens. 

### Motivation
Researchers do not know the data generating process for the data they are analyzing. This exercise creates a nonlinear data generating process that is too complex for simple modelling to capture, then compares OLS and Double Selection estimation.

### Objective:
* Create Monte Carlo experiement
* Compare distribution of coeffiecients estimated by OLS and Double Selection 
* Look at the effects of assumptions on these two models



## Step 1: Install packages 

In [1]:
rm(list=ls())
## This function will check if a package is installed, and if not, install it
pkgTest <- function(x) {
  if (!require(x, character.only = TRUE))
  {
    install.packages(x, dep = TRUE)
    if(!require(x, character.only = TRUE)) stop("Package not found")
  }
}

## These lines load the required packages
packages <- c("Hmisc", "hdm","mvtnorm")
lapply(packages, pkgTest)

Loading required package: Hmisc
Loading required package: lattice
Loading required package: survival
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:base’:

    format.pval, round.POSIXt, trunc.POSIXt, units

Loading required package: hdm
Loading required package: mvtnorm


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

## Step 2: Create data


Set seed for replication

In [2]:
set.seed(12345)

Set sample size

In [3]:
N <- 200

Set number of simulations

In [4]:
n.sims <- 40

Create matrix of 200 random normally distributed variables

In [5]:
x <- matrix(runif(N*50), nrow = N, ncol = 50)

Create polynomials

In [6]:
polyx<-cbind(x^2,x^3)

Create 5 bins for first 40 variables

In [7]:
nvar<-10
quants<-5
dumx<-matrix(nrow=N,ncol=quants*nvar)
  for(i in 1:nvar){
    indx<- factor(as.numeric(cut2(x[,i], g=quants)))
    dumx[,((quants*i)-(quants-1)):(quants*i)]<-model.matrix(~indx-1)
  }

Create treatment variable where half of the observations are treatment have are control

In [8]:
 Treatment<-sample(rep(0:1,N/2))

Create true y. Note:
* True effect of Treatment on y is 1
* Most x variables have no effect on y
* Some x variables have nonlinear effects on y

In [9]:
  y <- Treatment+x[,1] * 2+ x[,2] * 2.5 + 600*x[,3]-20*x[,3]^2-20*x[,3]^3+
                #3*exp(x[,4]+x[,10])+2*log(x[,6]+x[,10]+100)+ 
                600*x[,7]^2+30*x[,8]^2- 20*x[,8]^3+
                300*dumx[,(quants*5)]+90*dumx[,(quants*5)+1]+30*dumx[,(quants*5)+2]+
                100*dumx[,(quants*8)+1]+50*dumx[,(quants*8)+2]+70*dumx[,(quants*8)+3]+
                10*dumx[,(quants*9)]+60*dumx[,(quants*9)+3]-6*dumx[,(quants*9)+5]+
                rnorm(N)

## Step 3: Analyze Data

Estimate naive OLS model using only x variables and summarize

In [10]:
fit.lm <- lm(y~Treatment+x)
summary(fit.lm)


Call:
lm(formula = y ~ Treatment + x)

Residuals:
    Min      1Q  Median      3Q     Max 
-169.30  -66.12   -6.44   63.16  268.80 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -20.1101   109.9866  -0.183  0.85517    
Treatment     5.7128    16.8246   0.340  0.73468    
x1          -17.7822    28.3776  -0.627  0.53187    
x2           14.4849    28.5180   0.508  0.61227    
x3          598.3622    29.7448  20.117  < 2e-16 ***
x4            2.3085    27.7425   0.083  0.93379    
x5          319.1051    32.0652   9.952  < 2e-16 ***
x6          -78.6869    29.6003  -2.658  0.00872 ** 
x7          551.8391    27.7209  19.907  < 2e-16 ***
x8          -20.2293    28.6492  -0.706  0.48123    
x9          -73.2834    30.9538  -2.368  0.01920 *  
x10          12.0677    29.7788   0.405  0.68588    
x11          33.7327    29.5529   1.141  0.25553    
x12         -57.7439    28.5444  -2.023  0.04488 *  
x13          31.8338    29.3712   1.084  0.28020    
x14 

Estimate naive Double Selection model using only x variables and summarize

In [11]:
fit.ds <- rlassoEffect(x, y, Treatment, method = "double selection")
summary(fit.ds)

[1] "Estimates and significance testing of the effect of target variables"
   Estimate. Std. Error t value Pr(>|t|)
d1     1.151     14.174   0.081    0.935



Now assume some x variables have polynomial effects on y and estimate OLS with x and polyx variables

In [12]:
fit.lm.os <- lm(y~Treatment+x+polyx)
  summary(fit.lm.os)


Call:
lm(formula = y ~ Treatment + x + polyx)

Residuals:
    Min      1Q  Median      3Q     Max 
-83.656 -23.936  -1.085  22.635 105.297 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept)   148.156    282.552   0.524   0.6024  
Treatment      -9.788     20.361  -0.481   0.6329  
x1            -11.930    304.171  -0.039   0.9689  
x2           -221.436    399.319  -0.555   0.5818  
x3            816.653    317.934   2.569   0.0134 *
x4           -186.051    353.289  -0.527   0.6009  
x5            236.162    465.177   0.508   0.6140  
x6           -200.931    292.433  -0.687   0.4953  
x7             11.215    373.844   0.030   0.9762  
x8            392.652    387.501   1.013   0.3160  
x9            119.413    287.984   0.415   0.6802  
x10           258.702    370.835   0.698   0.4888  
x11           350.961    313.694   1.119   0.2688  
x12            78.661    424.961   0.185   0.8539  
x13            56.722    321.638   0.176   0.8608  
x14         

Assume some x variables have binned effects on y and estimate OLS 

In [13]:
fit.lm.os <- lm(y~Treatment+x+dumx+polyx)
  summary(fit.lm.os)


Call:
lm(formula = y ~ Treatment + x + dumx + polyx)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.33797 -0.09564 -0.00537  0.09139  0.37277 

Coefficients: (10 not defined because of singularities)
              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  344.09520   19.71657   17.452 1.19e-07 ***
Treatment      2.08709    0.98226    2.125  0.06633 .  
x1           -16.06694   20.12311   -0.798  0.44767    
x2             2.42491   13.56478    0.179  0.86257    
x3           578.81922   15.80207   36.629 3.38e-10 ***
x4           -77.99840   25.33961   -3.078  0.01516 *  
x5            36.05844   16.49822    2.186  0.06033 .  
x6           -14.92017   11.79164   -1.265  0.24137    
x7            26.33064   22.53185    1.169  0.27621    
x8            11.33347   10.09418    1.123  0.29410    
x9           -33.46767   22.30931   -1.500  0.17196    
x10           -8.67133   18.11126   -0.479  0.64491    
x11           31.08114   17.46702    1.779  0.11305    
x

Assume some x variables have binned and polynomial effects on y and estimate double selection

In [14]:
xds<-cbind(x,dumx,polyx)
  
fit.ds.os <- rlassoEffect(xds, y, Treatment, method = "double selection")
summary(fit.ds.os)

[1] "Estimates and significance testing of the effect of target variables"
   Estimate. Std. Error t value Pr(>|t|)    
d1    1.4340     0.3334   4.301  1.7e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



# Step 4: Monte Carlo

Each of these method give much different coefficients for Treatment. We will use a Monte Carlo to visualize the distribution of coefficients and measure the type 1 and type 2 errors for the tests.

### Create output matrices
Prefixes:
* betahat~ -- estimates of Treatment coefficient
* type1~ -- type one error(1), no type one error(0)
* type2~ -- type two error(1), no type two error(0), where false null is less than or equal to zero 

Suffixes:
* ~ols -- OLS estimation naive model
* ~ols.os -- OLS estimation overspecified model
* ~ds -- DS estimation naive model
* ~ds.os -- DS estimation overspecified model

In [15]:
betahat.ols <- matrix(NA,nrow=n.sims,ncol=3)
betahat.ds <- matrix(NA,nrow=n.sims,ncol=1)
betahat.ols.os <- matrix(NA,nrow=n.sims,ncol=3)
betahat.ds.os <- matrix(NA,nrow=n.sims,ncol=1)
type1.ols <- matrix(NA,nrow=n.sims,ncol=1)
type1.ds <- matrix(NA,nrow=n.sims,ncol=1)
type1.ols.os <- matrix(NA,nrow=n.sims,ncol=1)
type1.ds.os <- matrix(NA,nrow=n.sims,ncol=1)
type2.ols <- matrix(NA,nrow=n.sims,ncol=1)
type2.ds <- matrix(NA,nrow=n.sims,ncol=1)
type2.ols.os <- matrix(NA,nrow=n.sims,ncol=1)
type2.ds.os <- matrix(NA,nrow=n.sims,ncol=1)


##### Takes 15-20 minutes to run
* Run the Monte Carlo experiment using previously described data generating process

* Calculate type 1 and type 2 errors

In [60]:

for(j in 1:n.sims){
# Create data  
  x <- matrix(runif(N*50), nrow = N, ncol = 50)
  Treatment<-sample(rep(0:1,N/2))
  polyx<-cbind(x^2,x^3)
  nvar<-20
    quants<-5
    dumx<-matrix(nrow=N,ncol=5*nvar)
  for(i in 1:nvar){
    indx<- factor(as.numeric(cut2(x[,i], g=quants)))
    dumx[,((quants*i)-(quants-1)):(quants*i)]<-model.matrix(~indx-1)
  }
# Create y    
  y <- Treatment+x[,1] * 2+ x[,2] * 2.5 + 600*x[,3]-20*x[,3]^2-20*x[,3]^3+
                #3*exp(x[,4]+x[,10])+2*log(x[,6]+x[,10]+100)+ 
                600*x[,7]^2+30*x[,8]^2- 20*x[,8]^3+
                300*dumx[,(quants*5)]+90*dumx[,(quants*5)+1]+30*dumx[,(quants*5)+2]+
                100*dumx[,(quants*8)+1]+50*dumx[,(quants*8)+2]+70*dumx[,(quants*8)+3]+
                10*dumx[,(quants*9)]+60*dumx[,(quants*9)+3]-6*dumx[,(quants*9)+5]+
                rnorm(N)
#### Estimating true relationships
# Estimate naive OLS  
  fit.lm <- lm(y~Treatment+x)
  summary(fit.lm)
  
  b<-coef(summary(fit.lm))[,"Estimate"][2]
  s<-coef(summary(fit.lm))[,"Std. Error"][2]
  p<-coef(summary(fit.lm))[,"Pr(>|t|)"][2]
   
  t<-(b-1)/(s/sqrt(N))
# Store relavant statistics
  betahat.ols[j,]<-b
  
  if(t>1.96){
    type1.ols[j,1]<-1
  }
  else{
    type1.ols[j,1]<-0
  }
  
  if(b<0 | p>.05){
    type2.ols[j,1]<-1
  }
  else{
    type2.ols[j,1]<-0
  }

# Estimate naive DS  
  fit.ds <- rlassoEffect(x, y, Treatment, method = "double selection")
  summary(fit.ds)
  
  b.ds<-coef(summary(fit.ds))[,"Estimate."][1]
  s.ds<-coef(summary(fit.ds))[,"Std. Error"][1]
  p.ds<-coef(summary(fit.ds))[,"Pr(>|t|)"][1]
  
  t.ds<-(b.ds-1)/(s.ds/sqrt(N))
# Store relavant statistics  
  betahat.ds[j,]<-b.ds
  
  if(t.ds>1.96){
    type1.ds[j,1]<-1
  }
  else{
    type1.ds[j,1]<-0
  }
  if(b.ds<0 | p.ds>.05){
    type2.ds[j,1]<-1
  }
  else{
    type2.ds[j,1]<-0
  }
 
## Overspecify
# Estimate overspecified OLS 
  fit.lm.os <- lm(y~Treatment+x+dumx)
  summary(fit.lm.os)
  
  
  b.os<-coef(summary(fit.lm.os))[,"Estimate"][2]
  s.os<-coef(summary(fit.lm.os))[,"Std. Error"][2]
  p.os<-coef(summary(fit.lm.os))[,"Pr(>|t|)"][2]
  
  t.os<-(b.os-1)/(s.os/sqrt(N))
# Store relavant statistics  
  betahat.ols.os[j,]<-b.os
  
  if(t.os>1.96){
    type1.ols.os[j,1]<-1
  }
  else{
    type1.ols.os[j,1]<-0
  }
  
  if(b.os<0 | p.os>.05){
    type2.ols.os[j,1]<-1
  }
  else{
    type2.ols.os[j,1]<-0
  }

# Estimate overspecified DS
  xds<-cbind(x,dumx,polyx)
  
  fit.ds.os <- rlassoEffect(xds, y, Treatment, method = "double selection")
  summary(fit.ds.os)
  
  b.ds.os<-coef(summary(fit.ds.os))[,"Estimate."][1]
  s.ds.os<-coef(summary(fit.ds.os))[,"Std. Error"][1]
  p.ds.os<-coef(summary(fit.ds.os))[,"Pr(>|t|)"][1]
  
  t.ds.os<-(b.ds.os-1)/(s.ds.os/sqrt(N))
# Store relavant statistics  
  betahat.ds.os[j,]<-b.ds.os
  
  if(t.ds.os>1.96){
    type1.ds.os[j,1]<-1
  }
  else{
    type1.ds.os[j,1]<-0
  }
  
  if(b.ds.os<0 | p.ds.os>.05){
    type2.ds.os[j,1]<-1
  }
  else{
    type2.ds.os[j,1]<-0
  }
  print(paste0(j, ' of ',n.sims))
}

# Print type 1 and type 2 errors
print(paste0('type 1 error naive OLS rate = ',colMeans(type1.ols)))
print(paste0('type 1 error naive DS rate = ',colMeans(type1.ds)))
print(paste0('type 1 error overspecified OLS rate = ',colMeans(type1.ols.os)))
print(paste0('type 1 error overspecified DS rate = ',colMeans(type1.ds.os)))

print(paste0('type 2 error naive OLS rate = ',colMeans(type2.ols)))
print(paste0('type 2 error naive DS rate = ',colMeans(type2.ds)))
print(paste0('type 2 error overspecified OLS rate = ',colMeans(type2.ols.os)))
print(paste0('type 2 error overspecified DS rate = ',colMeans(type2.ds.os)))



[1] "1 of 40"
[1] "2 of 40"
[1] "3 of 40"
[1] "4 of 40"
[1] "5 of 40"
[1] "6 of 40"
[1] "7 of 40"
[1] "8 of 40"
[1] "9 of 40"
[1] "10 of 40"
[1] "11 of 40"
[1] "12 of 40"
[1] "13 of 40"
[1] "14 of 40"
[1] "15 of 40"
[1] "16 of 40"
[1] "17 of 40"
[1] "18 of 40"
[1] "19 of 40"
[1] "20 of 40"
[1] "21 of 40"
[1] "22 of 40"
[1] "23 of 40"
[1] "24 of 40"
[1] "25 of 40"
[1] "26 of 40"
[1] "27 of 40"
[1] "28 of 40"
[1] "29 of 40"
[1] "30 of 40"
[1] "31 of 40"
[1] "32 of 40"
[1] "33 of 40"
[1] "34 of 40"
[1] "35 of 40"
[1] "36 of 40"
[1] "37 of 40"
[1] "38 of 40"
[1] "39 of 40"
[1] "40 of 40"
[1] "type 1 error naive OLS rate = 0.675"
[1] "type 1 error naive DS rate = 0.525"
[1] "type 1 error overspecified OLS rate = 0.45"
[1] "type 1 error overspecified DS rate = 0.4"
[1] "type 2 error naive OLS rate = 0.975"
[1] "type 2 error naive DS rate = 0.95"
[1] "type 2 error overspecified OLS rate = 0.975"
[1] "type 2 error overspecified DS rate = 0.55"


Save density plots of coefficients

In [58]:
png(filename='MisspecDS.png')

plot (density(betahat.ds),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of naive Double Selection")

dev.off()

png(filename='MisspecOLS.png')

plot (density(betahat.ols),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of naive OLS")

dev.off()


png(filename='MisspecDSos.png')

plot (density(betahat.ds.os),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of overspecified Double Selection")

dev.off()

png(filename='MisspecOLSos.png')

plot (density(betahat.ols.os),col="red",main = "",xlab="Treatment Coefficient Estimate (True = 1)")
title(main = "Density Plot of overspecified OLS")

dev.off()

png 
  2

png 
  2

png 
  2

png 
  2

### Question 1

Which the four methods has the best distribution? Why? What assumptions of OLS and DS are violated in the naive models? 

### Question 2

Why does the overspecified OLS have a lower type 1 error rate? 

### Question 3

The type 2 error rate is calculated incorrectly. Describe how would you correct it? 

### Question 4

Create a correlation between the Treatment and the independent variables. How does this change the OLS and DS coefficents? What other methods can be used to deal with this specification?

### Question 5

Change the model or data generating process so that an assumption of double selection is violated. How does this affect the ability of the model to draw valid inference?